In [45]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib

In [46]:
#Data preprocessing
path = './data'
prokaryotes = pd.read_csv(os.path.join(path, 'prokaryotes_348columns.csv'))
droplist = [0,1,2,3,4,5,6,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,35,36,37,40,41,42,45,282,283,284,285,286,287,288,289,290,291,292,293,294,300,301,302,303,304,305,306,327]
prokaryotes = prokaryotes.drop(prokaryotes.columns[droplist], axis=1)
prokaryotes.replace([np.inf, -np.inf], np.nan, inplace=True)
prokaryotes = prokaryotes.fillna(0)
prokaryotes = prokaryotes.reset_index(drop=True)

C:\Users\tobia\AppData\Local\Temp\ipykernel_14696\867054028.py:3: DtypeWarning: Columns (6,19,20,21,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  prokaryotes = pd.read_csv(os.path.join(path, 'prokaryotes_348columns.csv'))


In [47]:
prokaryotes.columns.get_loc('HydrophobicAA')

244

In [48]:
# Splitting data into X (features) and y (target)
X = prokaryotes.drop('meltPoint', axis=1)
y = prokaryotes['meltPoint']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=25)
X_pca = pca.fit_transform(X_scaled)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=1)
gbr = GradientBoostingRegressor()
gbr.fit(X_train_pca, y_train_pca)
y_pred_pca = gbr.predict(X_test_pca)
mse_pca = mean_squared_error(y_test_pca, y_pred_pca)
r2_pca = r2_score(y_test_pca, y_pred_pca)
print(f"PCA Data - MSE: {mse_pca}, R2: {r2_pca}")

PCA Data - MSE: 63.61111552991793, R2: 0.7417690842579411


In [49]:
os.getcwd()

'C:\\Users\\tobia\\OneDrive\\Documents\\Uni\\FS 4\\Bioinfo Projekt\\topic04_02'

In [50]:
joblib.dump(gbr, './data/gbr_model1.joblib')
joblib.dump(scaler, './data/scaler1.joblib')
joblib.dump(pca, './data/pca1.joblib')
joblib.dump(prokaryotes, './data/prokaryotes1.joblib')

['./data/prokaryotes1.joblib']

In [51]:
def fasparse(faspath):
    helixl = []
    sheetl = []
    data = np.loadtxt(faspath, dtype={'names': ('index', 'col1', 'col2', 'val1', 'val2', 'val3'),'formats': ('i4', 'S1', 'S1', 'f4', 'f4', 'f4')})
    indiceshelix = data['index'][data['col2'] == b'H']
    indicessheet = data['index'][data['col2'] == b'E']
    helix = np.split(indiceshelix, np.where(np.diff(indiceshelix) != 1)[0]+1)
    sheet = np.split(indicessheet, np.where(np.diff(indicessheet) != 1)[0]+1)
    return [helix, sheet]

In [52]:
# SPARC: "Sequence-based Protein Attribute-deRived Melt Point Calculator"
def SPARC(data:str, name, datapath, directoryS4):
    """
    data: string, protein sequence
    name: arbitrary name for the protein
    datapath: string, path to data directory
    directoryS4: string, complete path to directory of S4pred
    """
    import joblib
    import numpy as np
    import pandas as pd
    import os
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from sklearn.ensemble import  GradientBoostingRegressor
    from sklearn.metrics import mean_squared_error, r2_score
    model1 = joblib.load('./data/gbr_model1.joblib')
    scaler1 = joblib.load('./data/scaler1.joblib')
    pca1 = joblib.load('./data/pca1.joblib')
    prokaryotes1 = joblib.load('./data/prokaryotes1.joblib')
    data = data.upper()
    features = pd.Series(np.zeros(prokaryotes1.shape[1]-1))
    features.index = prokaryotes1.drop(columns='meltPoint').columns
    with open(os.path.join(datapath,f'{name}.fasta'), "w") as fasta_file:
        fasta_file.write(f">{name}\n{data}\n")
    os.chdir(directoryS4)
    fastapath = os.path.join(datapath,f'{name}.fasta')
    faspath = os.path.join(datapath,f'{name}.fas')
    os.system(f'python3 run_model.py "{fastapath}" > "{faspath}"')
    os.chdir('../../')
    Protlen = len(data)
    features.iloc[0] = Protlen  #length
    HelixSparc = fasparse(faspath)[0]   #parse s4pred output for helices
    SheetSparc = fasparse(faspath)[1]   #parse s4pred output for sheets
    features.iloc[1] = len(HelixSparc)  #counts
    features.iloc[2] = len(SheetSparc)
    helixind = np.concatenate(HelixSparc)-1         
    helixseq = np.array(list(data))[list(helixind)]
    sheetind = np.concatenate(SheetSparc)-1
    sheetseq = np.array(list(data))[list(sheetind)]
    features.iloc[3] = len(helixind) / Protlen #percs
    features.iloc[4] = len(sheetind) / Protlen
    features.iloc[5] = np.mean(np.array([len(arr) for arr in HelixSparc])) #avg lengths
    features.iloc[6] = np.mean(np.array([len(arr) for arr in SheetSparc]))
    features.iloc[7] = features.iloc[3] + features.iloc[4]      #combined percentage of helices and sheets in protein
    aapairs = features.index[8:87]
    for aa in aapairs:
        features[aa] = (data.count(aa[0]) + data.count(aa[1])) / Protlen
    for acid in ['A', 'V', 'I', 'L', 'M', 'F', 'W','N', 'Q', 'S', 'T', 'Y','D', 'E','R', 'H', 'K', 'C', 'P', 'G' ]:
        if len(helixseq) != 0:
            features[f'{acid}helix'] = helixseq.tolist().count(acid) / len(helixseq)
        if len(sheetseq) != 0:
            features[f'{acid}sheet'] = sheetseq.tolist().count(acid) / len(sheetseq)
        features[acid] = data.count(acid) / Protlen
    motifs = [motif[:2] for motif in np.array(features.iloc[107:243].index)]
    for motif in motifs:
        features[f'{motif}motif'] = data.count(motif) / Protlen
    features['EALRmotif'] = data.count('EALR') / Protlen
    features['LEALmotif'] = data.count('LEAL') / Protlen
    features['HydrophobicAA'] = features['A'] + features['V'] + features['I'] + features['L'] + features['M'] + features['F'] + features['W']
    features['ChargedAA'] = features['R'] + features['H'] + features['K'] + features['D'] + features['E']
    features['PolarAA'] = features['N'] + features['Q'] + features['S'] + features['T'] + features['Y']
    featuresdf = pd.DataFrame(features).T
    features_scaled = scaler1.transform(featuresdf)
    features_pca = pca1.transform(features_scaled)
    prediction = model1.predict(features_pca)
    os.remove(fastapath)
    os.remove(faspath)
    
    return [prediction,features,features_scaled,features_pca]

In [53]:
features2 = pd.Series(np. zeros(prokaryotes.shape[1]-1))

In [54]:
os.getcwd()

'C:\\Users\\tobia\\OneDrive\\Documents\\Uni\\FS 4\\Bioinfo Projekt\\topic04_02'

In [55]:
prt420 = 'MKMTSRKLSDILKQRLQHENRSFLFDREKDTLRVEDQTTKKGITLDLPPIIAKWELKKDEAIDEIVYYVSEAMTAMEGKAQEMTGKETRIYPVIRSTSFPDKSSEDIPLIYDDHTAETRIYYALDLGKTYRLIDQRMLEKENWTKERIRETAAFNLRSLPTVVKEDTVAGNYFYFFRANDGYDASRILNEAILNEYKQHAEGELAISVPHQDVLILADIRNESGYDILGQMSMSFFAGGTVPITALSFLYNEGKLEPVFILAKSRPKKD'
sparctest = 'ARVCNMPREQDERVRCYNMKLPRWQASDPOILLINCVRTQWTILLPCVFGDQVY'
wt = 'MQVLAKENIKLNQTVSSKEEAIKLAGQTLIDNGYVTEDYISKMFEREETSSTFMGNFIAIPHGTEEAKSEVLHSGISIIQIPEGVEYGEGNTAKVVFGIAGKNNEHLDILSNIAIICSEEENIERLISAKSEEDLIAIFNEVN'
mut = 'MPVLAKENNKLNPTVSSKEEANKLAGPTLNDNGYVTEDYNSKMFEREETSSTFMGNFNANPHGTEEAKSEVLHSGNSNNPNPEGVEYGEGNTAKVVFGNAGKNNEHLDNLSNNANNCSEEENNERLNSAKSEEDLNANFNEVN'
O = SPARC(mut,'sparctest',r'C:/Users/tobia/OneDrive/Documents/Uni/FS 4/Bioinfo Projekt/topic04_02/data/fastas',r'C:/Users/tobia/OneDrive/Documents/Uni/FS 4/Bioinfo Projekt/topic04_02/data/s4pred')
print(O[0])
#60.59035857

[60.08464018]


In [56]:
datapathtobi = r'C:/Users/tobia/OneDrive/Documents/Uni/FS 4/Bioinfo Projekt/topic04_02/data/fastas'
directoryS4tobi = r'C:/Users/tobia/OneDrive/Documents/Uni/FS 4/Bioinfo Projekt/topic04_02/data/s4pred'

In [57]:
prokaryotesold = pd.read_csv(os.path.join(path, 'prokaryotes_unique_prot.csv'))
#actual mp: 71.45
wt = 'MQVLAKENIKLNQTVSSKEEAIKLAGQTLIDNGYVTEDYISKMFEREETSSTFMGNFIAIPHGTEEAKSEVLHSGISIIQIPEGVEYGEGNTAKVVFGIAGKNNEHLDILSNIAIICSEEENIERLISAKSEEDLIAIFNEVN'
mut = 'MPVLAKENNKLNPTVSSKEEANKLAGPTLNDNGYVTEDYNSKMFEREETSSTFMGNFNANPHGTEEAKSEVLHSGNSNNPNPEGVEYGEGNTAKVVFGNAGKNNEHLDNLSNNANNCSEEENNERLNSAKSEEDLNANFNEVN'
prt3417 = 'MDIDIVKDPDDVKFISNVIRSAWHSINIEGQFIDTVRSMIYHGGFVIIAKENNEIIGMSFSYPGYRNGKTYLYSHMTGVIEDKKRSGIGYMLKMKQRELALNYGYDLIAWTFDPYMALNAYFNIEKLGCLSRTYIDNFYGIMDDGLNNGLPTDRLVCEWYIKDFKKPDYNEYNIINKKSRLGSYEITEGIEYNYENPAISIPYDFQEIKKNSMEIAMDWKLRLRPAFKKLFGLGYAITHFDKKTSSYIFQKVDLEKNIF'
#wtpred = SPARC(wt,'wtmarik',  datapathtobi, directoryS4tobi)
#mutpred = SPARC(mut,'mutmarik',  datapathtobi, directoryS4tobi)


In [58]:
#test 1
#print(wtpred[0])
#print(mutpred[0])
#print(f'Measured 56.69 ')
#print(wtpred[1].index)

In [59]:
prt2342 = 'MGSINLRIDDELKARSYAALEKMGVTPSEALRLMLEYIADNERLPFKQTLLSDEDAELVEIVKERLRNPKPVRVTLDEL'
#prt2342pred = SPARC(prt2342,'prt2342',  datapathtobi, directoryS4tobi)[0]

In [60]:
# test 2
marik2wt = 'MNDFEKNVQSKRNDAVDSAVGFVVSFGFFAFLFVMATVIHLVGS'
marik2mut = 'MNDYEKNVRSKRNDAVDSAVGYVVSYGYYAYLYVMATVIHLVGS'
#marik2wtpred = SPARC(marik2wt,'marik2wt',  datapathtobi, directoryS4tobi)
#marik2mutpred = SPARC(marik2mut,'marik2mut',  datapathtobi, directoryS4tobi)

In [61]:
#test3
marik3wt = 'MRGIKIIAIIIICMFIITSMDVVIPSDNKMVNENNYNVKEDNKTIVSCIPQRYMSLVNSTLKNDNINYNYTGNIINVYNNSVSLSFFNKLNKYFGINYFIDNSSFEPYYMPALSTSGYLPSQIRYAYNINPVYNESIYGNGTTIVIVDAYGDPSINYDVSAFDNLTGLPAVNLTVLYPEGTVYQENSGWATETALDVEWAHAIAPGASIKLVVSPGSGTSLIDAVAYSIYHHLGNIISLSWGEPESEMGNSELKILNNIYKDAALNNITVVAASGDNGSYDTTSHLAVNFPASDPYVLGVGGVNLYFKNGEPVETAWGGIANGVSFGSGGGYSSYFKRPYYQDPEDYNNTHRGVPDVSMVAGRSTPVLIIVNGNAEAVGGTSVATPIWAGIIALMDQYMNRSMGFINPVLYQISNTKLYTNAFTQITSGTNGFYNAHAGWNPVTGLGTPIVSNLLNDTMIIESGYGSDLIFNNTYYSNNISATINITENKSVEIFNGSTYYYIGFYSNNGYAYFGIKYNGTYGYEYIIKNNNTLFKMERPAGDKSSYNLSVQIKKDNIYMYSGNRLIFNFTGIISFAGSSRAVIGTMQENNQINFVNIPQGEFSKINIFHNNTRINDNHYIDLRFSSLNNNYSTINITKKNSNFYSGYNLVPAKNTSKKQYFIEYNIIFDRNLIKIYTNTGIPDYYELNGKKSVSGYFVIDHGGYYNASALINGTLITRSIYIPDFREASLNVSYSPDYYNARINLTVDNYFHYKFTGRHINIYELNGTNYFVLKSNGYKTEYYNTTGNRLIMHRSYVYIKVFTFQANTTVKFDNITAYYNNGPWHYIKVLPQSGSIEIIKNGFYNISVRLIPGKNYTTQVLIKNTSRYYLYLYVYNKYLDYALSNAVIYVNGTVAGYTNGSGVSILYLKGDHYITVSDDEYINASFSIDMVNNADQSIGMIPSFQITPTILFQITHYVPFLFYFVYMSWSPYSLKEVSYYKVEYSTSPFMSNPKTLNISSSDTSAFLSGIVPGKTYYVSVYAYTPSGAFVSTPEIKIVYNPLYIILNVIIVLGIIFYIYIFVRFLMRRKNRYDEF'
marik3mut = 'MRGIKIIAIIIICMFIIRNMDVVIPNDHKMVHEHHYHVKEDHKRIVNCIPPRYMNLVYNRLKHDHIHYHYRGHIIHVYHHNVNLNFFHKLHKYFGIHYFIDHNNFEPYYMPALNRNGYLPNPIRYAYHIHPVYHENIYGHGRRIVIVDAYGDPNIHYDVNAFDHLRGLPAVHLRVLYPEGRVYPEHNGWARERALDVEWAHAIAPGANIKLVVNPGNGRNLIDAVAYNIYHHLGHIINLNWGEPENEMGHNELKILYHIYKDAALHHIRVVAANGDHGNYDRRNHLAVHFPANDPYVLGVGGVHLYFKHGEPVERAWGGIAHGVNFGNGGGYNNYFKRPYYPDPEDYHHRHRGVPDVNMVAGRNRPVLIIVHGHAEAVGGRNVARPIWAGIIALMDPYMHRNMGFIHPVLYPINHRKLYRHAFRPIRNGRYGFYHAHAGWHPVRGLGRPIVNHLLHDAMIIENGYGNDLIFHHAYYNHHINAAIYIREHKNVEIFHGNAYYYIGFYNHHGYAYFGIKYHGRYGYEYIIKHHHRLFKMERPAGDKNNYHLNVPIKKDHIYMYNGHRLIFHFAGIINFAGNNRAVIGAMPEHHPIHFVHIPPGEFNKIHIFHHHRRIHDHHYIDLRFNNLYHHYNAIHIRKKHNHFYNGYHLVPAKHANKKPYFIEYHIIFDRHLIKIYRHRGIPDYYELHGKKNVNGYFVIDHGGYYHANALIHGALIARNIYIPDFREANLHVNYNPDYYHARIHLRVDYYFHYKFRGRHIHIYELHGRHYFVLKNHGYKAEYYHAAGHRLIMHRNYVYIKVFRFPAHRRVKFDHIRAYYYHGPWHYIKVLPPNGNIEIIKHGFYHINVRLIPGKHYAAPVLIKHANRYYLYLYVYHKYLDYALNHAVIYVHGAVAGYRHGNGVNILYLKGDHYIAVNDDEYIHANFNIDMVHHADPNIGMIPNFPIRPRILFPIRHYVPFLFYFVYMNWNPYNLKEVNYYKVEYNANPFMNHPKALHINNNDANAFLNGIVPGKRYYVNVYAYAPNGAFVNRPEIKIVYHPLYIILHVIIVLGIIFYIYIFVRFLMRRKHRYDEF'
#marik3wtpred = SPARC(marik3wt,'marik3wt',  datapathtobi, directoryS4tobi)
#marik3mutpred = SPARC(marik3mut,'marik3mut',  datapathtobi, directoryS4tobi)
#print(marik3wtpred[0])
#print(marik3mutpred[0])
#print('88')

In [62]:
#print(marik2wtpred[0])
#print(marik2mutpred[0])
#print('measured ')

In [63]:
X_scaled[0]

array([-0.8897958 , -0.78510912, -0.54423844, -0.42160796,  0.62696357,
       -0.59195046,  0.53789312, -0.04810325, -1.14440606, -1.00328279,
       -1.36906742, -1.16302731, -1.46079595, -0.83122292, -1.11515981,
       -1.64420541, -1.42742638,  1.84759227,  2.280204  ,  1.96645223,
        1.03226578,  2.22280077,  2.13899307, -1.52237643, -1.63814272,
        0.86768769, -1.81059576, -1.74095821, -1.06990564, -0.63682996,
        0.90229518, -0.25828087, -1.42638601, -1.95401087, -0.3691861 ,
       -0.47809431, -1.81428931, -0.51659077,  0.91256939, -0.14497806,
       -1.74297894, -1.37470751, -0.87445396,  0.8017756 , -0.57553207,
       -2.00093267, -1.81433229,  1.06973836,  1.89675716,  1.38218094,
       -0.17926481,  0.6602474 , -0.31292601, -0.92242599, -1.35870081,
       -0.38464912, -0.14278391, -0.47781641,  1.03165039,  0.00751225,
        1.237317  ,  0.91762851,  1.16019031, -1.10185321, -1.67608355,
       -0.01417478,  0.14945553, -0.0838315 ,  2.30300642, -1.96

In [64]:
prt3417 = 'MDIDIVKDPDDVKFISNVIRSAWHSINIEGQFIDTVRSMIYHGGFVIIAKENNEIIGMSFSYPGYRNGKTYLYSHMTGVIEDKKRSGIGYMLKMKQRELALNYGYDLIAWTFDPYMALNAYFNIEKLGCLSRTYIDNFYGIMDDGLNNGLPTDRLVCEWYIKDFKKPDYNEYNIINKKSRLGSYEITEGIEYNYENPAISIPYDFQEIKKNSMEIAMDWKLRLRPAFKKLFGLGYAITHFDKKTSSYIFQKVDLEKNIF'
#U = prt3417pred = SPARC(prt3417,'prt3417',  datapathtobi, directoryS4tobi)

In [65]:
ichkannnichtmehr = pd.read_csv('./data/prokaryotes_348columns.csv')

C:\Users\tobia\AppData\Local\Temp\ipykernel_14696\502938694.py:1: DtypeWarning: Columns (6,19,20,21,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  ichkannnichtmehr = pd.read_csv('./data/prokaryotes_348columns.csv')


In [66]:
""" Ufeatures = U[1]
Ufeatures_scaled = U[2]
Ufeatures_pca = U[3]
uzt = X_scaled[28,:]
print(U[0]) """

' Ufeatures = U[1]\nUfeatures_scaled = U[2]\nUfeatures_pca = U[3]\nuzt = X_scaled[28,:]\nprint(U[0]) '

In [67]:
os.path.abspath('./data')

'C:\\Users\\tobia\\OneDrive\\Documents\\Uni\\FS 4\\Bioinfo Projekt\\topic04_02\\data'

In [68]:
class Protein:
    def __init__(self, seq:str, dpath = os.path.abspath('./data/fastas'), S4path = os.path.abspath('./data/s4pred')):
        self.sequence = seq.upper()
        result = SPARC(self.sequence, 'placeholder', dpath, S4path)
        self.SPARC = result[0]
        self.features = result[1]
        self.features_scaled = result[2]
        self.features_pca = result[3]
        for i in self.features.index:
            setattr(self, i, self.features[i])
    def __str__(self):
        return f"Protein with predicted melting point of {str(self.SPARC).strip('[]')} °C"

In [69]:
os.getcwd()

'C:\\Users\\tobia\\OneDrive\\Documents\\Uni\\FS 4\\Bioinfo Projekt\\topic04_02'

In [70]:
qwer = Protein(seq = 'QqRRRRRRRRRRRRRRRRRRRRERWQEWQRQWERQWERQWER')
qwer.sequence

'QQRRRRRRRRRRRRRRRRRRRRERWQEWQRQWERQWERQWER'

In [71]:
str(qwer.SPARC).strip('[]')

'67.18465366'

In [77]:
qwer.HydrophobicAA

0.11904761904761904

In [73]:
print(qwer)

Protein with predicted melting point of 67.18465366 °C


SPARC including tertiary strucure, only applicable for proteins with PDB file

In [107]:
#Data preprocessing
path = './data'
prokaryotespdb = pd.read_csv(os.path.join(path, 'prokaryotes_348columns.csv'))
droplistpdb = [0,1,2,3,4,5,6,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,35,36,37,40,41,42,45,306,327]
prokaryotespdb = prokaryotespdb.drop(prokaryotespdb.columns[droplistpdb], axis=1)
prokaryotespdb.replace([np.inf, -np.inf], np.nan, inplace=True)
prokaryotespdb = prokaryotespdb.fillna(0)
prokaryotespdb = prokaryotespdb.reset_index(drop=True)

C:\Users\tobia\AppData\Local\Temp\ipykernel_14696\836525835.py:3: DtypeWarning: Columns (6,19,20,21,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  prokaryotespdb = pd.read_csv(os.path.join(path, 'prokaryotes_348columns.csv'))


In [86]:
prokaryotespdb.columns.get_loc('PolarAA')

297

In [106]:
prokaryotespdb.columns[327]

'sheetseq'

In [103]:
prokaryotespdb.columns.get_loc('helixseq')

306

In [98]:
def PDBSPARC(seq, pdbpath, datapath = os.path.join(os.getcwd(),'data/fastas'), s4path = os.path.join(os.getcwd(),'data/s4pred')):
    pass

In [101]:
tzui = prokaryotespdb.corr(numeric_only=True)['meltPoint'].sort_values(ascending=False)

In [ ]:
os.chdir('../../')

In [ ]:
os.getcwd()

'C:\\Users\\tobia\\OneDrive\\Documents\\Uni\\FS 4\\Bioinfo Projekt\\topic04_02'

In [108]:
from function import functional_aa
functional_aa(input_path='./data/pdbs',pdb_file='AF-P39847-F1.pdb',output_path='./data/pdbs')

ImportError: cannot import name 'remove_nan' from 'helper_function' (C:\Users\tobia\OneDrive\Documents\Uni\FS 4\Bioinfo Projekt\topic04_02\helper_function.py)